# SolidWorks MCP Server Tutorial

This notebook demonstrates how to interact with the SolidWorks MCP server for CAD automation.

In [ ]:
# Import required libraries
import asyncio
import json
from pathlib import Path

# Import MCP components
from src.tools.solidworks_tools import SolidWorksTools
from src.context_builder.knowledge_base import SolidWorksKnowledgeBase
from src.solidworks_adapters.factory import AdapterFactory

## 1. Initialize Components

In [ ]:
# Initialize knowledge base
kb = SolidWorksKnowledgeBase()

# Initialize tools
tools = SolidWorksTools(knowledge_base=kb)

# Initialize adapter factory
factory = AdapterFactory()

# Get adapter for current SolidWorks version
adapter = factory.get_best_adapter()

print(f"Using adapter for SolidWorks {adapter.version}")

## 2. Connect to SolidWorks

In [ ]:
# Connect to SolidWorks
connected = await adapter.connect()
print(f"Connected to SolidWorks: {connected}")

## 3. Open a Model

In [ ]:
# Open a part file
result = await tools.execute(
    "open_model",
    {"file_path": "C:/SolidWorks/Models/example.sldprt"},
    adapter
)

print(json.dumps(result, indent=2))

## 4. Analyze Model Features

In [ ]:
# Get all features
features_result = await tools.execute(
    "get_features",
    {},
    adapter
)

# Display feature summary
if features_result["success"]:
    features = features_result["features"]
    print(f"Total features: {len(features)}")
    print("\nFeature types:")
    for ftype, count in features_result["summary"]["feature_types"].items():
        print(f"  - {ftype}: {count}")

## 5. Modify Dimensions

In [ ]:
# Modify a dimension
dim_result = await tools.execute(
    "modify_dimension",
    {
        "feature_name": "Boss-Extrude1",
        "dimension_name": "D1@Boss-Extrude1",
        "value": 25.0
    },
    adapter
)

if dim_result["success"]:
    print(f"Dimension changed from {dim_result.get('original_value')} to {dim_result['new_value']}")

## 6. Knowledge Base Queries

In [ ]:
# Find similar operations
similar_ops = await kb.find_similar_operations(
    "modify dimension extrusion",
    n_results=3
)

print("Similar operations found:")
for op in similar_ops:
    print(f"- {op['operation']}: {op.get('success', 'Unknown')}")

## 7. Export Model

In [ ]:
# Export to STEP format
export_result = await tools.execute(
    "export_model",
    {
        "output_path": "C:/Exports/example.step",
        "format": "STEP"
    },
    adapter
)

if export_result["success"]:
    print(f"Exported to: {export_result['output_path']}")
    print(f"File size: {export_result.get('file_size_mb', 0)} MB")

## 8. Analyze Operation History

In [ ]:
# Get operation statistics
stats = await kb.analyze_operation_patterns()

print(f"Total operations: {stats['total_operations']}")
print(f"Success rate: {stats['success_rate']:.1%}")
print("\nMost common operations:")
for op_type, count in list(stats['operation_types'].items())[:5]:
    print(f"  - {op_type}: {count}")

## 9. Cleanup

In [ ]:
# Disconnect from SolidWorks
await adapter.disconnect()
print("Disconnected from SolidWorks")